In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from collections import Counter
import tensorflow as tf

import os
import pickle
import re
from tensorflow.python.ops import math_ops
from tensorflow.keras import layers, models
import time


In [ ]:
#https://www.tensorflow.org/learn?hl=zh-cn
# TensorFlow2.0中高级API全部整合到tf.keras仓库下了，因此我们要查找高级Api就去tf.kears仓库下找
# 主要API模块有一下：
# activations: 包含当前主流的激活函数，可以直接通过该API进行激活函数的调用
# applications: 模块中是已经进行预训练好的神经网络模型，可以直接进行使用或者进行迁移学习
# backend: 包含了keras后台的一些基础API接口，用于实现高级的API或者自己构建神经网络
# datasets: 包含了常用的公开数据训练及，可以直接进行使用(需要翻墙，或下载好放到.keras/datasets目录下),有CIFAR100,Boston Housing等
# layers：包含众多的神经网络层接口,可以通过直接调用API的方式实现不同的网络层结构
# losses: 包含了常用的损失函数，可以根据实际的需求直接进行调用
# optimizers：包含主流的优化器（如ADAM）
# preprocessing:包含了数据处理的一些方法，分为图片数据处理，语言序列处理，文本数据处理等：比如NLP常用的pad_sequences等
# regularizers:包含了常用的正则化方法，包括L1，L2正则化方法
# wrappers:是一个keras模型的包装器，当我们需要进行跨框架迁移时，可以使用该API接口提供与其他框架的兼容性

# models:有Sequential有顺序的的顺序模型，和Model自定义模型（可以指定任意形式的输入与输出），

In [11]:
#layers的相关学习：tf.keras.layers.

#Input输入层可以接受对应shape的num.array()作为输入：最终创建的模型的输入要和Input层的shape一致
layers.Input(shape=())#<tf.Tensor 'input_5:0' shape=(None,) dtype=float32>
layers.Input(shape=(5,5)) #<tf.Tensor 'input_4:0' shape=(None, 5, 5) dtype=float32>

#Embedding嵌入层：嵌入式向量可以方便寻找向量与向量之间的联系（相似度），one-hot互相垂直无法表示相似度
# 0000维的onehot表征向量变成300的嵌入式向量，每句话有10个词；
# 例如：[9876,5684,7568,3547,7896,7524,8965，2456，125，85]是一句话
# 嵌入后就会变为一个10*32的新矩阵，每一行的32的值就对应代表了上面每个词的嵌入，
layers.Embedding(input_dim=10000, output_dim=300, input_length=10)

#Flatten打平层，可以将shape == (None, 64, 32, 32)打平为(None, 65536)
layers.Flatten()


# concatenate 链接层
#layer1,layer2, layer3, layer4的shape都为（10258，32）链接后变为10258，128）
#axis参数可以指定是链接第几层，具体使用时可以尝试（注意模型中究竟应该链接的层数）
layer1 = layer2 = layer3 = layer4 = layers.Input(shape=(32,))
layers.concatenate(
        [layer1,layer2, layer3, layer4]
)#<tf.Tensor 'concatenate_1/Identity:0' shape=(None, 128) dtype=float32>

# Dense全连接层：输出层的神经元个数为200个
# 2D input with shape `(batch_size, input_dim)`：output would have shape `(batch_size, units)`
# N-D tensor input with shape: `(batch_size, ..., input_dim)`:output shape:(batch_size, ..., units)
layers.Dense(units=200)






In [44]:
# tf.reduce_XXX这个也是对layer进行操作的
reduce_input_layer = layers.Input(shape=(3,5,))
# 当axis=1,keepdims=True：<tf.Tensor 'Sum_1:0' shape=(None, 1, 5) dtype=float32>
reduce_sum_layer1 = tf.reduce_sum(reduce_input_layer,axis=1,keepdims=True)
# 当 axis=1,keepdims=False：<tf.Tensor 'Sum_3:0' shape=(None, 5) dtype=float32>
reduce_sum_layer2 = tf.reduce_sum(reduce_input_layer,axis=1,keepdims=False)
# 当axis=2,keepdims=True：<tf.Tensor 'Sum_6:0' shape=(None, 3, 1) dtype=float32>
reduce_sum_layer3 = tf.reduce_sum(reduce_input_layer,axis=2,keepdims=True)
# 当 axis=2,keepdims=False：<tf.Tensor 'Sum_14:0' shape=(None, 3) dtype=float32>
reduce_sum_layer4 = tf.reduce_sum(reduce_input_layer,axis=2,keepdims=False)

#boolean类型的reduce
reduce_bool_input_layer = layers.Input(shape=(3,5,),dtype=tf.bool)
#<tf.Tensor 'All:0' shape=(None, 3, 1) dtype=bool>
reduce_layer = tf.reduce_all(reduce_bool_input_layer,axis=2,keepdims=True)

#<tf.Tensor 'Min_1:0' shape=(None, 1, 5) dtype=float32>
reduce_layer = tf.reduce_min(reduce_input_layer,axis=1,keepdims=True)

reduce_layer

<tf.Tensor 'Min_1:0' shape=(None, 1, 5) dtype=float32>

In [20]:
# models中的自定义Model:可以任意指定输入与输出
input_layer1 = layers.Input(shape=(32,))
input_layer2 = layers.Input(shape=(32,))
input_layer3 = layers.Input(shape=(32,))
input_layer4 = layers.Input(shape=(32,))
output_layer = layers.concatenate(
        [input_layer1,input_layer2, input_layer3, input_layer4]
)
model = models.Model(inputs=[input_layer1, input_layer2, input_layer3, input_layer4],
                                      outputs=output_layer)
#model_user_feature.predict()
